In [ ]:
import zipfile
import os

# Define the path to your uploaded .zip file
zip_file_path = 'UCF_and_Shanghai.zip'

# Extract the contents to a folder
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('Data')  # Folder to extract to

In [ ]:
import torch
from torch.utils.data import Dataset
import numpy as np
import os
import random

class Normal_Loader(Dataset):
    """
    is_train = 1 <- train, 0 <- test
    """

    def __init__(self, is_train=1, path='C:\\Users\\Data\\UCF-Crime\\'):
        super(Normal_Loader, self).__init__()
        self.is_train = is_train
        self.path = path
        if self.is_train == 1:
            data_list = os.path.join(path, 'train_normal.txt')
            with open(data_list, 'r') as f:
                self.data_list = f.readlines()
        else:
            data_list = os.path.join(path, 'test_normalv2.txt')
            with open(data_list, 'r') as f:
                self.data_list = f.readlines()
            random.shuffle(self.data_list)
            self.data_list = self.data_list[:-10]
    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        if self.is_train == 1:
            rgb_npy = np.load(os.path.join(self.path+'all_rgbs', self.data_list[idx][:-1]+'.npy'))
            flow_npy = np.load(os.path.join(self.path+'all_flows', self.data_list[idx][:-1]+'.npy'))
            concat_npy = np.concatenate([rgb_npy, flow_npy], axis=1)
            return concat_npy
        else:
            name, frames, gts = self.data_list[idx].split(' ')[0], int(self.data_list[idx].split(' ')[1]), int(self.data_list[idx].split(' ')[2][:-1])
            rgb_npy = np.load(os.path.join(self.path+'all_rgbs', name + '.npy'))
            flow_npy = np.load(os.path.join(self.path+'all_flows', name + '.npy'))
            concat_npy = np.concatenate([rgb_npy, flow_npy], axis=1)
            return concat_npy, gts, frames

class Anomaly_Loader(Dataset):
    """
    is_train = 1 <- train, 0 <- test
    """
    def __init__(self, is_train=1, path='C:\\Users\\Data\\UCF-Crime\\'):
        super(Anomaly_Loader, self).__init__()
        self.is_train = is_train
        self.path = path
        if self.is_train == 1:
            data_list = os.path.join(path, 'train_anomaly.txt')
            with open(data_list, 'r') as f:
                self.data_list = f.readlines()
        else:
            data_list = os.path.join(path, 'test_anomalyv2.txt')
            with open(data_list, 'r') as f:
                self.data_list = f.readlines()

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        if self.is_train == 1:
            rgb_npy = np.load(os.path.join(self.path+'all_rgbs', self.data_list[idx][:-1]+'.npy'))
            flow_npy = np.load(os.path.join(self.path+'all_flows', self.data_list[idx][:-1]+'.npy'))
            concat_npy = np.concatenate([rgb_npy, flow_npy], axis=1)
            return concat_npy
        else:
            name, frames, gts = self.data_list[idx].split('|')[0], int(self.data_list[idx].split('|')[1]), self.data_list[idx].split('|')[2][1:-2].split(',')
            gts = [int(i) for i in gts]
            rgb_npy = np.load(os.path.join(self.path+'all_rgbs', name + '.npy'))
            flow_npy = np.load(os.path.join(self.path+'all_flows', name + '.npy'))
            concat_npy = np.concatenate([rgb_npy, flow_npy], axis=1)
            return concat_npy, gts, frames

if __name__ == '__main__':
    loader2 = Normal_Loader(is_train=0)
    print(len(loader2))
    #print(loader[1], loader2[1])

140


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class Learner(nn.Module):
    def __init__(self, input_dim=2048, drop_p=0.5):
        super(Learner, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
        self.drop_p = drop_p
        self.weight_init()
        self.vars = nn.ParameterList()

        for i, param in enumerate(self.classifier.parameters()):
            self.vars.append(param)

    def weight_init(self):
        for layer in self.classifier:
            if type(layer) == nn.Linear:
                nn.init.xavier_normal_(layer.weight)

    def forward(self, x, vars=None):
        if vars is None:
            vars = self.vars
        x = F.linear(x, vars[0], vars[1])
        x = F.relu(x)
        x = F.dropout(x, self.drop_p, training=self.training)
        x = F.linear(x, vars[2], vars[3])
        x = F.dropout(x, self.drop_p, training=self.training)
        x = F.linear(x, vars[4], vars[5])
        return torch.sigmoid(x)

    def parameters(self):
        """
        override this function since initial parameters will return with a generator.
        :return:
        """
        return self.vars

In [ ]:
import torch
import torch.nn.functional as F

def MIL(y_pred, batch_size, is_transformer=0):
    loss = torch.tensor(0.)
    loss_intra = torch.tensor(0.)
    sparsity = torch.tensor(0.)
    smooth = torch.tensor(0.)

    if is_transformer == 0:
        y_pred = y_pred.view(batch_size, -1)
    else:
        y_pred = torch.sigmoid(y_pred)

    for i in range(batch_size):
        anomaly_index = torch.randperm(30)
        normal_index = torch.randperm(30)

        y_anomaly = y_pred[i, :32][anomaly_index]
        y_normal  = y_pred[i, 32:][normal_index]

        y_anomaly_max = torch.max(y_anomaly)  # anomaly
        y_anomaly_min = torch.min(y_anomaly)

        y_normal_max = torch.max(y_normal)  # normal
        y_normal_min = torch.min(y_normal)

        loss += F.relu(1. - y_anomaly_max + y_normal_max)

        sparsity += torch.sum(y_anomaly) * 0.00008
        smooth += torch.sum((y_pred[i, :31] - y_pred[i, 1:32])**2) * 0.00008

    # Average the total loss
    loss = (loss + sparsity + smooth) / batch_size

    return loss


In [ ]:
class CustomLearner(nn.Module):
    def __init__(self, input_dim=2048, hidden_dim=512, drop_p=0.5):
        super(CustomLearner, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc3 = nn.Linear(hidden_dim // 2, 1)
        self.dropout = nn.Dropout(drop_p)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
import torch
from torch.utils.data import DataLoader
from loss import *
from dataset import *
import os
from sklearn import metrics
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Load datasets
normal_train_dataset = Normal_Loader(is_train=1)
normal_test_dataset = Normal_Loader(is_train=0)

anomaly_train_dataset = Anomaly_Loader(is_train=1)
anomaly_test_dataset = Anomaly_Loader(is_train=0)

# Create DataLoaders
normal_train_loader = DataLoader(normal_train_dataset, batch_size=30, shuffle=True)
normal_test_loader = DataLoader(normal_test_dataset, batch_size=1, shuffle=True)

anomaly_train_loader = DataLoader(anomaly_train_dataset, batch_size=30, shuffle=True)
anomaly_test_loader = DataLoader(anomaly_test_dataset, batch_size=1, shuffle=True)

# Set device to CPU
device = 'cpu'  # Ensure it's set to CPU if CUDA is not available

# Model definition (using the custom learner)
# Commented out the original Learner model and using the custom one
# model = Learner(input_dim=2048, drop_p=0.0).to(device)
model = CustomLearner(input_dim=2048, drop_p=0.5).to(device)

# Optimizer, scheduler, and criterion
optimizer = torch.optim.Adagrad(model.parameters(), lr = 1e-3, weight_decay = 1e-4)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[25, 50])
criterion = MIL

# Training function
def train(epoch):
    print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (normal_inputs, anomaly_inputs) in enumerate(zip(normal_train_loader, anomaly_train_loader)):
        inputs = torch.cat([anomaly_inputs, normal_inputs], dim=1)
        batch_size = inputs.shape[0]
        inputs = inputs.view(-1, inputs.size(-1)).to(device)  # Ensure inputs are on the correct device (CPU)
        outputs = model(inputs)
        loss = criterion(outputs, batch_size)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print('loss = {}'.format(train_loss / len(normal_train_loader)))
    scheduler.step()

# Testing function
def test_abnormal(epoch):
    model.eval()
    auc = 0
    with torch.no_grad():
        for i, (data, data2) in enumerate(zip(anomaly_test_loader, normal_test_loader)):
            inputs, gts, frames = data
            inputs = inputs.view(-1, inputs.size(-1)).to(device)  # Ensure inputs are on the correct device (CPU)
            score = model(inputs)
            score = score.cpu().detach().numpy()
            score_list = np.zeros(frames[0])
            step = np.round(np.linspace(0, frames[0] // 16, 33))

            for j in range(32):
                score_list[int(step[j]) * 16:(int(step[j + 1])) * 16] = score[j]

            gt_list = np.zeros(frames[0])
            for k in range(len(gts) // 2):
                s = gts[k * 2]
                e = min(gts[k * 2 + 1], frames)
                gt_list[s - 1:e] = 1

            # Handle second dataset
            inputs2, gts2, frames2 = data2
            inputs2 = inputs2.view(-1, inputs2.size(-1)).to(device)  # Ensure inputs are on the correct device (CPU)
            score2 = model(inputs2)
            score2 = score2.cpu().detach().numpy()
            score_list2 = np.zeros(frames2[0])
            step2 = np.round(np.linspace(0, frames2[0] // 16, 33))

            for kk in range(32):
                score_list2[int(step2[kk]) * 16:(int(step2[kk + 1])) * 16] = score2[kk]

            gt_list2 = np.zeros(frames2[0])

            score_list3 = np.concatenate((score_list, score_list2), axis=0)
            gt_list3 = np.concatenate((gt_list, gt_list2), axis=0)

            fpr, tpr, thresholds = metrics.roc_curve(gt_list3, score_list3, pos_label=1)
            auc += metrics.auc(fpr, tpr)
        print('auc = ', auc / 140)
    return auc / 140


# Training and Testing Loop
for epoch in range(0, 75):
    train(epoch)
    test_abnormal(epoch)


'''
early_stopping_patience = 20

best_auc = 0
patience_counter = 0
for epoch in range(0, 75):
    train(epoch)
    auc_score = test_abnormal(epoch)

    # Check if AUC improves
    if auc_score > best_auc:
        best_auc = auc_score
        patience_counter = 0  # Reset patience if improvement
        torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        patience_counter += 1

    if patience_counter >= early_stopping_patience:
        print("Early stopping triggered")
        break  # Stop training if no improvement

'''



Epoch: 0
loss = 0.8889971596223337
auc =  0.7944221009688623

Epoch: 1
loss = 0.4421863964310399
auc =  0.8031596388008561

Epoch: 2
loss = 0.26872747225893867
auc =  0.825570671093132

Epoch: 3
loss = 0.2237130375923934
auc =  0.8328133223024011

Epoch: 4
loss = 0.19905463302576984
auc =  0.831737231655539

Epoch: 5
loss = 0.19361005944234352
auc =  0.82320402905321

Epoch: 6
loss = 0.16938918132196973
auc =  0.8365234075875427

Epoch: 7
loss = 0.15802361471233545
auc =  0.8262522790451542

Epoch: 8
loss = 0.15536913261921317
auc =  0.8228922223214337

Epoch: 9
loss = 0.1376445279629142
auc =  0.8354248963008114

Epoch: 10
loss = 0.13707745792689147
auc =  0.828318032023623

Epoch: 11
loss = 0.13552221097052097
auc =  0.8377244043028831

Epoch: 12
loss = 0.1334553823840839
auc =  0.8238470264465108

Epoch: 13
loss = 0.12647519229600826
auc =  0.8265030638944969

Epoch: 14
loss = 0.13572110961777745
auc =  0.8339316118229521

Epoch: 15
loss = 0.13024270720779896
auc =  0.8263186827823

'\nearly_stopping_patience = 20\n\nbest_auc = 0\npatience_counter = 0\nfor epoch in range(0, 75): \n    train(epoch)\n    auc_score = test_abnormal(epoch)\n\n    # Check if AUC improves\n    if auc_score > best_auc:\n        best_auc = auc_score\n        patience_counter = 0  # Reset patience if improvement\n        torch.save(model.state_dict(), \'best_model.pth\')  # Save the best model\n    else:\n        patience_counter += 1\n\n    if patience_counter >= early_stopping_patience:\n        print("Early stopping triggered")\n        break  # Stop training if no improvement\n\n'